# Read SIFT 1 million 

In [8]:
using HDF5
using BenchmarkTools
using Distances
using LoopVectorization
using SIMD
using Clustering
using ProgressMeter
using StaticArrays

In [9]:
Sys.cpu_info()[1].model

"Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz"

In [10]:
path = joinpath(homedir(), "Datasets", "SIFT1M",
                "sift-128-euclidean.hdf5")

"/Users/davidbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5"

In [11]:
f = h5open(path, "r")

🗂️ HDF5.File: (read-only) /Users/davidbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5
├─ 🏷️ distance
├─ 🔢 distances
├─ 🔢 neighbors
├─ 🔢 test
└─ 🔢 train

In [12]:
X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
neighbors = read(f["neighbors"])
distances = read(f["distances"])

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(neighbors)
@show size(distances)

size(X_tr_vecs) = (128, 1000000)
size(X_te_vecs) = (128, 10000)
size(neighbors) = (100, 10000)
size(distances) = (100, 10000)


(100, 10000)

## Clustering.jl

Create the pqcodes for the training points

In [13]:
n_features = size(X_tr_vecs,1)
n_features

128

In [14]:
unique_values = sort(unique(X_tr_vecs))

213-element Vector{Float32}:
   0.0
   1.0
   2.0
   3.0
   4.0
   5.0
   6.0
   7.0
   8.0
   9.0
  10.0
  11.0
  12.0
   ⋮
 202.0
 204.0
 205.0
 206.0
 207.0
 208.0
 210.0
 211.0
 212.0
 214.0
 216.0
 218.0

In [15]:
n_clusters = 16
R_per_feature = []
@showprogress for j in  1:n_features
    R = kmeans(X_tr_vecs[[j],:], n_clusters; maxiter=200)
    push!(R_per_feature, R)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:02:36


In [16]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[0.49666122, 137.62012, 11.801707, 65.22103, 107.01379, 33.809605, 5.2500305, 125.62709, 25.328632, 43.77468, 97.311646, 87.348625, 76.83099, 116.23137, 54.338634, 18.393635]
(16, 128)


In [17]:
P = hcat([r.centers[:] for r in R_per_feature]...);
println(P[:,1])
println(size(P))

Float32[0.49666122, 137.62012, 11.801707, 65.22103, 107.01379, 33.809605, 5.2500305, 125.62709, 25.328632, 43.77468, 97.311646, 87.348625, 76.83099, 116.23137, 54.338634, 18.393635]
(16, 128)


In [18]:
@benchmark P = hcat([r.centers[:] for r in R_per_feature]...)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  12.808 μs … 119.344 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     14.019 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.906 μs ±   7.082 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅██▆▃▂▂▃▂▁                                                   ▁
  ███████████▇▇▆▆▅▆▅▅▄▅▄▅▄▄▃▄▄▄▄▅▅▆▅▄▄▄▄▅▄▃▅▅▆▆▆▆▇▇▇▇▇▆▇▇▇▇▇▇▇ █
  12.8 μs       Histogram: log(frequency) by time      45.9 μs <

 Memory estimate: 28.47 KiB, allocs estimate: 137.

Another way to construct the matrix would be

In [19]:
function build_prototype_matrix(R, n_clusters, n_features)
    
    prototypes = Array{Float32}(undef, n_clusters, n_features);
    for j in 1:n_features
        prototypes[:,j] .= vec(R_per_feature[j].centers)
    end
    return prototypes
end

build_prototype_matrix (generic function with 1 method)

In [20]:
@benchmark P = build_prototype_matrix(R_per_feature, n_clusters, n_features)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  57.854 μs … 419.584 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     63.972 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   72.375 μs ±  21.272 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅███▇▅▄▃▃▂▁▁▁▂▂▂▂▃▂▂▂▂▁▁ ▁   ▁▁  ▁                           ▂
  █████████████████████████████████████████▇▇█▇▇▆▇▅▆▆▅▆▆▄▆▅▆▄▆ █
  57.9 μs       Histogram: log(frequency) by time       158 μs <

 Memory estimate: 44.12 KiB, allocs estimate: 897.

In [21]:
P2 = build_prototype_matrix(R_per_feature, n_clusters, n_features);

In [22]:
isapprox(P, P2)

true

We can generate the PQcodes for the data as follows:

In [23]:
PQcodes = hcat([Int8.(r.assignments[:]) for r in R_per_feature]...)';

In [24]:
size(PQcodes)

(128, 1000000)

Here each vector prototype is in fact a single scalar (vecause subvectors have a single coordinate)

In [25]:
size(P)

(16, 128)

We need a method to, given a vector and a collection of prototypes per feature, find which are the closest

In [26]:
query = X_te_vecs[:,1];

In [27]:
closest_prototypes = Array{UInt8}(undef, n_features, 1);

In [28]:
typeof(typemax(Float32))

Float32

In [29]:
@inline function seuclidean(x::T,y::T) where {T}
    aux::T = (x -y)
    return aux * aux
end

function find_closest_coordinate(dist, 
                                 value::T,
                                 vector) where T
    best_coordinate = 1
    min_distance::T = typemax(T)
    for (j,x) in enumerate(vector)
        current_dist = dist(x, value)
        if current_dist < min_distance
            best_coordinate = j
            min_distance = current_dist
        end
    end
    return best_coordinate
end

find_closest_coordinate (generic function with 1 method)

In [30]:
find_closest_coordinate(seuclidean, Float32(48.), query)
print(query)

Float32[1.0, 3.0, 11.0, 110.0, 62.0, 22.0, 4.0, 0.0, 43.0, 21.0, 22.0, 18.0, 6.0, 28.0, 64.0, 9.0, 11.0, 1.0, 0.0, 0.0, 1.0, 40.0, 101.0, 21.0, 20.0, 2.0, 4.0, 2.0, 2.0, 9.0, 18.0, 35.0, 1.0, 1.0, 7.0, 25.0, 108.0, 116.0, 63.0, 2.0, 0.0, 0.0, 11.0, 74.0, 40.0, 101.0, 116.0, 3.0, 33.0, 1.0, 1.0, 11.0, 14.0, 18.0, 116.0, 116.0, 68.0, 12.0, 5.0, 4.0, 2.0, 2.0, 9.0, 102.0, 17.0, 3.0, 10.0, 18.0, 8.0, 15.0, 67.0, 63.0, 15.0, 0.0, 14.0, 116.0, 80.0, 0.0, 2.0, 22.0, 96.0, 37.0, 28.0, 88.0, 43.0, 1.0, 4.0, 18.0, 116.0, 51.0, 5.0, 11.0, 32.0, 14.0, 8.0, 23.0, 44.0, 17.0, 12.0, 9.0, 0.0, 0.0, 19.0, 37.0, 85.0, 18.0, 16.0, 104.0, 22.0, 6.0, 2.0, 26.0, 12.0, 58.0, 67.0, 82.0, 25.0, 12.0, 2.0, 2.0, 25.0, 18.0, 8.0, 2.0, 19.0, 42.0, 48.0, 11.0]

In [31]:
@benchmark find_closest_coordinate(seuclidean, Float32(48.), query)


BenchmarkTools.Trial: 10000 samples with 755 evaluations.
 Range (min … max):  161.446 ns … 484.444 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     169.287 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   175.063 ns ±  20.058 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅▃ ▆█▆▃▂▂▂▁                                                   ▁
  ████████████████████████▇█▇██▇██▇▇▇▆▇▆▆▆▅▇▆▆▅▆▅▄▄▅▅▅▅▄▄▅▅▃▄▄▄ █
  161 ns        Histogram: log(frequency) by time        266 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [32]:
@assert 3 == find_closest_coordinate(seuclidean, 2., [3.,5,2,1])

@assert 4 == find_closest_coordinate(seuclidean, 1., [3.,5,2,1])

In [33]:
function encode(dist, vector, prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    @inbounds for (j,x) in enumerate(vector)
            closest_prototypes[j] = find_closest_coordinate(dist, x, prototypes[:,j])
    end
    return closest_prototypes
end

encode (generic function with 1 method)

In [34]:
find_closest_coordinate(sqeuclidean, query[1], P[:,1])

1

In [35]:
encode(sqeuclidean, query, P)

128×1 Matrix{Int8}:
  1
 15
 12
  2
  9
  7
 13
  4
  8
 12
  7
 15
 15
  ⋮
  9
 12
  1
  2
  6
  4
  1
 13
  8
 11
 12
 12

In [36]:
@benchmark encode(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):  7.986 μs … 36.839 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.342 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.141 μs ±  2.309 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▇▅▃▂▂▁▁▁▁                            ▁        ▁          ▁
  ████████████▇▇▇▆▇▇▆▇▅▇▇▇▇▇█▆▇▇▇▇▇▆▇▆▆▆████▆▆▅▅▆▇██▇▇▇▅▄▅▄▄ █
  7.99 μs      Histogram: log(frequency) by time     17.6 μs <

 Memory estimate: 16.19 KiB, allocs estimate: 129.

In [37]:
function encode_fast(dist, vector::Array{T}, prototypes::Array{T}) where T
    n_clusters, n_features = size(prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k,j],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_fast (generic function with 1 method)

In [38]:
(query[1] .- P[:,1]) .* (query[1] .- P[:,1])

16-element Vector{Float32}:
     0.25334996
 18665.057
   116.67688
  4124.341
 11238.923
  1076.4702
    18.06276
 15531.912
   591.8823
  1829.6733
  9275.933
  7456.085
  5750.3394
 13278.269
  2845.01
   302.53854

In [39]:
println(find_closest_coordinate(sqeuclidean, query[1], P[:,1]))
println(find_closest_coordinate(sqeuclidean, query[2], P[:,2]))

1
15


In [40]:
encode_fast(sqeuclidean, query, P)[1:4]

4-element Vector{Int8}:
  1
 15
 12
  2

In [41]:
@assert isapprox(encode(sqeuclidean, query, P),
                 encode_fast(sqeuclidean, query, P))

In [42]:
@benchmark encode_fast(sqeuclidean, query, P)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.116 μs …  14.892 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.164 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.318 μs ± 604.987 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▅▃▁▁                                                      ▁
  ███████████▇▇▇█▇▇█▇▆▆▆▇▇▆▅▅▆▅▅▅▅▅▆▄▅▅▄▅▅▅▄▅▅▅▄▄▄▅▅▄▄▅▅▄▅▄▅▄ █
  2.12 μs      Histogram: log(frequency) by time      5.15 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [43]:
function compute_ADC(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters
            ADC_table[p,j] = dist(query[j], @view prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC (generic function with 1 method)

In [44]:
size(P)

(16, 128)

In [45]:
function SEuclidean0(x, query)
    res = zero(eltype(x))
    @inbounds   for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return res
end

SEuclidean0 (generic function with 1 method)

In [46]:
@benchmark compute_ADC(query, P, SEuclidean0)

BenchmarkTools.Trial: 6167 samples with 1 evaluation.
 Range (min … max):  640.650 μs … 218.623 ms  ┊ GC (min … max): 0.00% … 99.65%
 Time  (median):     730.490 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   802.631 μs ±   2.777 ms  ┊ GC (mean ± σ):  4.40% ±  1.27%

    ▇██▇▆▄▃▁ ▁                                                   
  ▂▅██████████▇▇▇▆▆▅▅▄▄▃▄▃▃▂▃▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  641 μs           Histogram: frequency by time         1.25 ms <

 Memory estimate: 272.25 KiB, allocs estimate: 10501.

In [47]:
adc_table = compute_ADC(query, P, SEuclidean0);

In [202]:
function compute_ADC_fast(query, prototypes, dist)
    n_clusters, n_features = size(P)
    
    ADC_table = Array{Float32}(undef, n_clusters, n_features)
    
    for j in 1:n_features
        for p in 1:n_clusters
            
            ADC_table[p,j] = dist(query[j],  prototypes[p,:] )
        end
    end
    return  ADC_table
end

compute_ADC_fast (generic function with 1 method)

In [203]:
@benchmark compute_ADC_fast(query, P, SEuclidean0)

BenchmarkTools.Trial: 4390 samples with 1 evaluation.
 Range (min … max):  856.282 μs …   6.059 ms  ┊ GC (min … max):  0.00% … 79.34%
 Time  (median):     975.866 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.132 ms ± 717.565 μs  ┊ GC (mean ± σ):  11.48% ± 14.16%

  ██▇▆▅▃▂▁                                                      ▁
  ████████▇▆▆▆▅▅▃▁▁▃▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▄▅▆▅▇▇▇▇▇▇▇▇ █
  856 μs        Histogram: log(frequency) by time       5.05 ms <

 Memory estimate: 1.48 MiB, allocs estimate: 18693.

In [214]:
function adc_dist(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds @fastmath for j in eachindex(x_code)
        res+= adc_table[x_code[j], j]
    end
    return res
end

adc_dist (generic function with 1 method)

In [215]:
query_code = encode_fast(sqeuclidean, query, P)
x_code = PQcodes[:,1];
adc_table = compute_ADC(query, P, SEuclidean0);
adc_dist(query_code, x_code,  adc_table)

473463.53f0

In [51]:
@benchmark adc_dist(query_code, x_code,  adc_table)

BenchmarkTools.Trial: 10000 samples with 981 evaluations.
 Range (min … max):  55.314 ns … 478.893 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     62.940 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   67.565 ns ±  15.739 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂  ▄▃██▄▂▂▂▁       ▁▁  ▁                                     ▁
  █▆▄██████████▇▇▇█▇▇██████████▇▇█▆▆▆▆▆▆█▆▆▅▆▆▆▆▅▅▅█▇▄▅▄▅▆▅▅▅▅ █
  55.3 ns       Histogram: log(frequency) by time       131 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [52]:
x = X_tr_vecs[:,1];
@benchmark SEuclidean0(x, query)

BenchmarkTools.Trial: 10000 samples with 800 evaluations.
 Range (min … max):  156.260 ns …  1.843 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     158.925 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   168.599 ns ± 42.504 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁                                          ▁
  █████████████████████████▇█▇█▇▇▇▇▇▆▆▇▆▆▆▆▆▅▆▅▅▆▆▅▄▄▄▅▄▅▅▄▄▄▄ █
  156 ns        Histogram: log(frequency) by time       258 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

##  Share prototypes across features

https://groups.google.com/g/julia-users/c/xBcQRebyi_o



In [148]:
n_clusters = 32
#R_shared = kmeans(Matrix(vec(X_tr_vecs)'), n_clusters; maxiter=200)
R_shared = kmeans(X_tr_vecs[[1],:], n_clusters; maxiter=200)

KmeansResult{Matrix{Float32}, Float32, Int64}(Float32[9.485008 65.47876 … 23.976738 13.485671], [3, 32, 3, 28, 3, 30, 3, 12, 24, 4  …  16, 1, 1, 15, 15, 22, 7, 18, 3, 21], Float32[0.05256829, 0.26455688, 0.05256829, 0.26367188, 0.5940128, 0.21679688, 0.05256829, 0.20184326, 1.0723343, 0.28729248  …  1.0510254, 0.265213, 0.23522949, 2.144043, 0.28686523, 3.8291016, 0.8599529, 0.2175293, 0.05256829, 0.037109375], [36020, 14055, 202536, 44210, 14355, 22456, 82724, 14273, 18570, 16753  …  26892, 63822, 13070, 10719, 36023, 32024, 18610, 20469, 29447, 29555], [36020, 14055, 202536, 44210, 14355, 22456, 82724, 14273, 18570, 16753  …  26892, 63822, 13070, 10719, 36023, 32024, 18610, 20469, 29447, 29555], 1.2977016f6, 3, true)

In [152]:
length(unique(R_shared.centers))

32

In [159]:
println(maximum(R_shared.centers))
println(maximum(X_tr_vecs[[1],:]))

142.66107
172.0


In [166]:
P_shared = R_shared.centers

1×32 Matrix{Float32}:
 9.48501  65.4788  0.229278  20.4641  …  70.424  48.4661  23.9767  13.4857

Now we need a new encoding function that uses a single vector of prototypes

In [186]:
function encode_fast_shared(dist, vector::Array{T}, prototypes::Array{T}) where T
    
    n_features = length(vector)
    n_clusters = length(prototypes)
    closest_prototypes = Array{Int8}(undef, n_features, 1);
    
    @inbounds @fastmath for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(prototypes[k],x )
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_fast_shared (generic function with 1 method)

In [187]:
@benchmark encode_fast_shared(sqeuclidean, query, P_shared)

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  3.932 μs …  14.147 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.959 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.058 μs ± 449.637 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▄                                                         ▁
  ███▇▇▆▇▆▆▅▅▆▇▆▄▅▅▅▇▆▅▇▆▅▅▄▃▄▅▂▄▄▃▄▄▄▄▄▄▄▄▄▃▅▄▃▅▅▄▄▄▄▄▅▂▄▅▃▄ █
  3.93 μs      Histogram: log(frequency) by time      6.63 μs <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [208]:
function compute_ADC_shared(query, prototypes, dist)
    n_clusters = length(prototypes)
    ADC_table = Array{Float32}(undef, n_clusters)
    
    @inbounds  for p in 1:n_clusters            
        ADC_table[p] = dist(query[p],  prototypes[p] )
    end
    return  ADC_table
end

compute_ADC_shared (generic function with 1 method)

In [216]:
query_code = encode_fast_shared(sqeuclidean, query, P_shared)
x_code = PQcodes[:,1];
adc_table_shared = compute_ADC_shared(query, P_shared, SEuclidean0);
#adc_dist(query_code, x_code,  adc_table)

In [212]:
@benchmark adc_table = compute_ADC_shared(query, P_shared, SEuclidean0)

BenchmarkTools.Trial: 10000 samples with 985 evaluations.
 Range (min … max):  54.020 ns …   2.673 μs  ┊ GC (min … max):  0.00% … 93.48%
 Time  (median):     56.176 ns               ┊ GC (median):     0.00%
 Time  (mean ± σ):   73.164 ns ± 142.865 ns  ┊ GC (mean ± σ):  11.84% ±  5.94%

  ▇█▅▄▂▁  ▁▂▂      ▂▂▁ ▃▂▁              ▁                      ▁
  █████████████████████████▇▆▇▇▆▆▆▆▅▆▆▅████▇▇▆▇▆▄▅▄▄▂▅▅▄▅▅▅▅▅▅ █
  54 ns         Histogram: log(frequency) by time       146 ns <

 Memory estimate: 192 bytes, allocs estimate: 1.

In [225]:
function adc_dist_shared(query_code, x_code,  adc_table)
    res = zero(eltype(adc_table))
    @inbounds  @simd for j in eachindex(x_code)
        res+= adc_table[x_code[j]]
    end
    return res
end

adc_dist_shared (generic function with 1 method)

In [226]:
@benchmark adc_dist_shared($query_code, $x_code,  $adc_table_shared)

BenchmarkTools.Trial: 10000 samples with 992 evaluations.
 Range (min … max):  38.895 ns … 167.028 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     38.920 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   39.520 ns ±   4.430 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅  ▁                                                        ▁
  █████▇▇▇█▇▆▆▇▆▆▆▅▅▄▅▅▅▃▅▆▅▅▅▁▃▁▁▅▄▆▆▄▃▄▃▅▁▅▁▄▅▄▃▃▃▄▁▃▅▃▁▃▄▁▄ █
  38.9 ns       Histogram: log(frequency) by time        54 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [228]:
x_code

128-element Vector{Int8}:
  1
 12
  3
 13
 12
  8
 12
  6
 14
 15
  3
  2
 12
  ⋮
 10
  6
 16
 14
 10
  2
  2
  1
  8
 16
 13
  4

## Euclidean  Squared Distance

In [ ]:
function SEuclidean(X, query)
    d = (query .- X) .* (query .- X)
    return sum(d, dims=1)
end

In [ ]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

In [ ]:
query = X_te_vecs[:,1];

In [ ]:
@benchmark SEuclidean(X_te_vecs, query)

In [ ]:
@benchmark SEuclidean_2(X_te_vecs, query)

In [ ]:
@benchmark SEuclidean(X_tr_vecs, query)

In [ ]:
@benchmark SEuclidean_2(X_tr_vecs, query)

## Finding top k distances (and their ids)


The first naive thing we can do consist on computing all distances and then sorting them to get the top k closest vectors to the query vector

In [ ]:
function top_k_ids(X, query)
    distances = SEuclidean_2(X, query)
    top_k_indices = sortperm(distances)
    return top_k_indices
end

In [ ]:
@benchmark top_k_ids(X_te_vecs, query)[1:10]

A slightly better approach consist on using `partialsortperm` to simply sort a subset of the distances vector.

In [ ]:
function top_k_ids_2(X, query, k)
    distances = SEuclidean_2(X, query)
    top_k_indices = partialsortperm(distances, 1:k)
    return top_k_indices
end

In [ ]:
@benchmark top_k_ids_2(X_te_vecs, query, 10)

### Storing top k distances in a priority queue

A better alternative consists on using a priority queue. This. queue will keep only k distances in memory (no need to store all distances between the query point and all possible candidates).

In [ ]:
sort!([1,54,3,24,10])

In [ ]:
a = [1,2,3,4,5]

In [ ]:
function SEuclidean_3(X, query, top_k)
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res/n_features
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end
            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end

    end
    return result
end

In [ ]:
@benchmark SEuclidean_3(X_tr_vecs, query, 10) 

In [ ]:
@benchmark top_k_ids_2(X_tr_vecs, query, 10)

In [ ]:
function SEuclidean_4(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    @inbounds @fastmath for m in top_k:n_examples
        res = zero(eltype(X))
        @simd for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end


In [ ]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10) 

In [ ]:
X_tr_200k = X_tr_vecs[:,1:200_000]
@benchmark SEuclidean_4(X_tr_200k, query, 10) 

In [ ]:
SEuclidean_4(X_tr_vecs, query, 10)

In [ ]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_2(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        res = zero(eltype(X))
        @turbo  for j in 1:n_features
            aux = (query[j] - X[j,m])
            res += aux * aux
        end
        dist = res
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

In [ ]:
@benchmark SEuclidean_5(X_tr_200k, query, 10) 

# Comparison with Distances.jl


Distances.jl provides the method 'pairwise' to compute pairwise distances between two matrices.

In [ ]:
@show typeof(query)
@show size(query)
@show typeof(X_tr_vecs)
@show size(X_tr_vecs)

In [ ]:
v1 = [1,2,3]
v2 = [0,0,1]

y = [0,0,2]
X = [v1 v2]

In [ ]:
function SEuclidean_5(X, query, top_k)
    
    result = sort(SEuclidean_5(X[:, 1:top_k], query))
    n_features, n_examples = size(X)
    
    for m in top_k:n_examples
        dist = zero(eltype(X))
        @turbo  for j in 1:n_features
            dist = (query[j] - X[j,m])
            dist += aux * aux
        end
        
        # see if current mse is in the top pile
        if dist < result[end]
            j = top_k 
            #revisamos la lista resultado de izq a derecha
            while dist < result[j-1]
                j = j-1
                if j == 1
                    break
                end
            end            
            result[j+1:end] .= result[j:end-1]
            result[j] = dist
         end
    end
    return result
end

In [ ]:
function SEuclidean_2(X, query)
    n_features, n_examples = size(X)
    result = zeros(eltype(X), n_examples)
    for m in 1:n_examples
        res = zero(eltype(X))
        for j in 1:n_features
            aux = (query[j] .- X[j,m])
            res += aux * aux
        end
        result[m] = res
    end
    return result
end

In [ ]:
y_colvec = [y y][:,[1]]
query_mat = [query query][:,[1]];

In [ ]:
SEuclidean_2(X, y)

In [ ]:
pairwise(SqEuclidean(), y_colvec, X)

In [ ]:
@benchmark pairwise(SqEuclidean(), query_mat, X_tr_vecs)

In [ ]:
@benchmark SEuclidean_2(X_tr_vecs, query)

In [ ]:
@benchmark SEuclidean_4(X_tr_vecs, query, 10)

We can verify that the sorted distances by `SEuclidean_4` are the same as the results provided with `pairwise` from `Distances.jl`

In [ ]:
res2 = SEuclidean_4(X_tr_vecs, query, 10);
res = pairwise(SqEuclidean(), query_mat, X_tr_vecs)
res = sort(res, dims=2)[1:10]
res == res2

## Parallel implementation

# SIMD tests

In [ ]:

function find_val_in_array_simd(x::Array{T}, val::T) where {T}
    n_simd = 64
    last_pos_simd_chunk = length(x)-n_simd
    @inbounds for i in 1:n_simd:last_pos_simd_chunk
        vec_i = vload(Vec{n_simd, T}, x, i)
        sum_equality = sum(vec_i == val)
        if sum_equality >0
            return true
        end
    end

    @inbounds for i in last_pos_simd_chunk:length(x)
        if x[i] == val
            return true
        end
    end

    return false
end

In [ ]:
function jdotavx(a, b)
    s = zero(eltype(a))
    @turbo for i ∈ eachindex(a, b)
        s += a[i] * b[i]
    end
    s
end